In [1]:
import numpy as np
import pandas as pd
from skopt import gp_minimize, gbrt_minimize 
from skopt.plots import plot_convergence, plot_objective
from rl4greencrab.envs.green_crab_monthly_env import greenCrabMonthEnv
from rl4greencrab.envs.green_crab_monthly_env_norm import greenCrabMonthEnvNormalized
from rl4greencrab import evaluate_agent, multiConstAction, simulator, environment_simulation, plot_selected_sizes, agent_action_plot
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
from huggingface_hub import hf_hub_download
from stable_baselines3 import PPO, TD3
from sb3_contrib import TQC, RecurrentPPO
from rl4greencrab import LipschitzPPO
from rl4greencrab import plot_agent
import ray

2025-07-28 00:12:35,683	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.8.5 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/opt/conda/lib/python3.12/site-packages/gymnasium/spaces/box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


In [2]:
config = {
    "w_mort_scale" : 600,
    "growth_k": 0.70,
    'random_start':True,
    'var_penalty_const': 0
    # "curriculum": True
}

## greencrab population change over times for each size

In [ ]:
agent = multiConstAction(env=env, action=np.array([0, 0, 0]))
constantData = environment_simulation(env, agent)
constantDataDF = pd.DataFrame(constantData)

In [ ]:
crab_pop_df = pd.DataFrame(constantDataDF[constantDataDF['rep']==0]['crab_pop'].tolist(), columns=[f'crab_pop_{i}' for i in range(len(constantDataDF['crab_pop'][0]))])
crab_pop_df['t'] = constantDataDF['t']

In [ ]:
# Create a multi-select widget for crab size columns
crab_pop_columns = [col for col in crab_pop_df.columns if col.startswith('crab_pop_')]
size_selector = widgets.SelectMultiple(
    options=crab_pop_columns,
    value=[crab_pop_columns[0]],  # Default selection
    description='Sizes',
    disabled=False
)
# Create an interactive plot
interact(lambda selected_sizes: plot_selected_sizes(crab_pop_df, selected_sizes), selected_sizes=size_selector);

## greencrab size distribution at certain time t

In [ ]:
def pop_df_gen(dataDF):
    df = pd.DataFrame(dataDF[dataDF['rep']==0]['crab_pop'].tolist(), 
                      columns=[f'crab_pop_{i}' for i in range(len(dataDF['crab_pop'][0]))])
    df.columns = [i for i in range(len(df.columns))]
    return df

def plot_size_distribution(dataDF, time_step):
    crab_pop_df = pop_df_gen(dataDF)
    plt.figure(figsize=(8, 5))
    plt.bar(crab_pop_df.columns, crab_pop_df.loc[time_step], color='blue', alpha=0.7)
    plt.ylim(0, 20000)
    plt.title(f'Crab Size Distribution at {time_step}')
    plt.xlabel('Crab Size')
    plt.ylabel('Density')
    plt.grid(True)
    plt.show()

def interact_size_plot(dataDF):
    return interact(
        lambda time_step: plot_size_distribution(dataDF, time_step),
         time_step=dataDF.index
    );

In [ ]:
agent = multiConstAction(env=env, action=np.array([0.0, 0, 0.0]))
noInterventionData = environment_simulation(env, agent)
noInterventionDataDF = pd.DataFrame(constantData)
interact_size_plot(noInterventionDataDF)

In [ ]:
agent = multiConstAction(env=env, action=np.array([83.87232800633504, 596.3225575635984, 14.882297944474463]))
constantData = environment_simulation(env, agent)
constantDataDF = pd.DataFrame(constantData)
interact_size_plot(constantDataDF)

In [ ]:
eval_env = greenCrabMonthEnvNormalized(config)
ppoAgent = PPO.load("../saved_agents/PPO_gcmenorm", device="cpu")
ppoDataDF = pd.DataFrame(environment_simulation(eval_env, ppoAgent))
interact_size_plot(ppoDataDF)

In [ ]:
eval_env = greenCrabMonthEnvNormalized(config)
tqcAgent = TQC.load(f"varRatio0.3_new/TQC_gcmenorm", device="cpu")
tqcDataDF = pd.DataFrame(environment_simulation(eval_env, tqcAgent))
interact_size_plot(tqcDataDF)

In [ ]:
df = tqcDataDF
df[df.rep == 0].iloc[:, :].plot(x='t', y=['act0', 'act1', 'act2'], title="Actions over Time")

In [ ]:
crab_pop_df = pop_df_gen(tqcDataDF)
crab_pop_df.iloc[:, :][8].plot()
plt.xlabel('t')
plt.ylabel('size_5')

In [ ]:
act_state_df = df[df.rep == 0].merge(right=crab_pop_df, left_index=True, right_index=True).drop(labels = 'crab_pop', axis=1)

In [ ]:
plt.scatter(x=act_state_df[11], y=act_state_df['act1'])
plt.xlabel('size_5')
plt.ylabel('act0')
# act_state_df.plot(x=5, y='act0')

## Overlap Policy Size Distribution

In [ ]:
dataDF_list = [noInterventionDataDF, constantDataDF, ppoDataDF]

In [ ]:
def plot_size_distr_overlap(dataDF_list, time_step):
    plt.figure(figsize=(8, 5))
    for i, df in enumerate(dataDF_list):
        crab_pop_df = pop_df_gen(df)
        plt.bar(
            crab_pop_df.columns,
            crab_pop_df.loc[time_step],
            alpha=0.5,
            label=f"DataFrame {i+1}"
        )
    plt.legend()
    plt.title(f'Crab Size Distribution at {time_step}')
    plt.xlabel('Crab Size')
    plt.ylabel('Density')
    plt.grid(True)
    plt.show()

def interact_size_plot(dataDF_list):
    return interact(
        lambda time_step: plot_size_distr_overlap(dataDF_list, time_step),
         time_step=pd.DataFrame(dataDF_list[0]).index
    );

In [ ]:
interact_size_plot(dataDF_list)

## Model Reward Distributions

In [ ]:
evalEnv =  greenCrabMonthEnvNormalized(config)

In [ ]:
path = "../saved_agents"
td3Agent = TD3.load(f"TD3_gcmenorm", device="cpu")
ppoAgent = PPO.load(f"PPO_gcmenorm", device="cpu")
tqcAgent = TQC.load(f"TQC_gcmenorm", device="cpu")
recurrentPPOAgent = RecurrentPPO.load("RecurrentPPO_gcmenorm_256_1_var0", device="cpu")
lppoAgent = LipschitzPPO.load("LipschitzPPO_gcmenorm_0.001_50", device="cpu")

In [ ]:
td3Agent = TD3.load(f"varianceRatio0/TD3_gcmenorm", device="cpu")
ppoAgent = PPO.load(f"varianceRatio0/PPO_gcmenorm", device="cpu")
tqcAgent = TQC.load(f"varRatio0.3_new/TQC_gcmenorm", device="cpu")
recurrentPPOAgent = RecurrentPPO.load("varianceRatio0.3/RecurrentPPO_gcmenorm_256_1_varR0.3", device="cpu")
lppoAgent = LipschitzPPO.load("varRatio0.3_new/LipschitzPPO_gcmenorm_0.1_1", device="cpu")

In [ ]:
def evaluateConstAct(x):
    env = greenCrabMonthEnvNormalized(config)
    agent = multiConstAction(env=env, action=np.array(x))
    # rewards = simulator(env, agent).simulate()
    # out = np.mean(rewards)
    rewards = evaluate_agent(agent=agent, ray_remote=True, gpu=False).evaluate(n_eval_episodes=50, return_episode_rewards=True)
    m_reward = np.mean(rewards)
    return -m_reward, rewards

# normalized constant action
def normalized_action(act, max_act):
    max_action = max_act
    constant_act = np.array(act)
    return constant_act * 2 / max_action - 1 

In [ ]:
# normalized constant action
def normalized_action(act, max_act):
    max_action = max_act
    constant_act = np.array(act)
    return constant_act * 2 / max_action - 1 

In [ ]:
ray.init()
norm_act = normalized_action([0.0, 0.0, 154.3889141214999], 3000)
m_reward, const_rews = evaluateConstAct(norm_act)
ray.shutdown()

In [ ]:
N_EPS = 30
ray.init()
ppo_rew = evaluate_agent(agent=ppoAgent, env=evalEnv, ray_remote=True).evaluate(n_eval_episodes=N_EPS)
td3_rew = evaluate_agent(agent=td3Agent, env=evalEnv, ray_remote=True).evaluate(n_eval_episodes=N_EPS)
tqc_rew = evaluate_agent(agent=tqcAgent, env=evalEnv, ray_remote=True).evaluate(n_eval_episodes=N_EPS)
reppo_rew = evaluate_agent(agent=recurrentPPOAgent, env=evalEnv, ray_remote=True).evaluate(n_eval_episodes=N_EPS)
lppo_rew = evaluate_agent(agent=lppoAgent, env=evalEnv, ray_remote=True).evaluate(n_eval_episodes=N_EPS)

In [ ]:
N_EPS = 200

ppo_rews = evaluate_agent(agent=ppoAgent, env=evalEnv, ray_remote=True).evaluate(n_eval_episodes=N_EPS, return_episode_rewards=True)
td3_rews = evaluate_agent(agent=td3Agent, env=evalEnv, ray_remote=True).evaluate(n_eval_episodes=N_EPS, return_episode_rewards=True)
tqc_rews = evaluate_agent(agent=tqcAgent, env=evalEnv, ray_remote=True).evaluate(n_eval_episodes=N_EPS, return_episode_rewards=True)
reppo_rews = evaluate_agent(agent=recurrentPPOAgent, env=evalEnv, ray_remote=True).evaluate(n_eval_episodes=N_EPS, return_episode_rewards=True)
lppo_rews = evaluate_agent(agent=lppoAgent, env=evalEnv, ray_remote=True).evaluate(n_eval_episodes=N_EPS, return_episode_rewards=True)

In [ ]:
import ray
ray.shutdown()

In [ ]:
plt.hist(ppo_rews, alpha=0.6, label='PPO')
plt.hist(td3_rews, alpha=0.6, label='TD3')
plt.hist(tqc_rews, alpha=0.6, label='TQC')
plt.hist(reppo_rews , alpha=0.6, label='RecurentPPO')
plt.hist(lppo_rews , alpha=0.6, label='LipschitzPPO')
plt.hist(const_rews, alpha=0.6, label='Constant')

# # Mark means with vertical lines
plt.axvline(np.mean(ppo_rews), color='blue', linestyle='dashed', linewidth=2, label='PPO Mean')
plt.axvline(np.mean(td3_rews), color='orange', linestyle='dashed', linewidth=2, label='TD3 Mean')
plt.axvline(np.mean(tqc_rews), color='green', linestyle='dashed', linewidth=2, label='TQC Mean')
plt.axvline(np.mean(const_rews), color='red', linestyle='dashed', linewidth=2, label='Constant Mean')

plt.legend()
plt.xlabel("Reward")
plt.ylabel("Frequency")
plt.title("Reward Distribution Comparison")
plt.show()

In [ ]:
from rl4greencrab import agent_rew_vs_constant_rew_plot

In [ ]:
models = {
    'PPO': ppo_rews,
    'TD3': td3_rews,
    'TQC': tqc_rews,
    'RecurrentPPO': reppo_rews,
    'LipschitzPPO': lppo_rews
}
constant_rewards = const_rews
agent_rew_vs_constant_rew_plot(models, constant_rewards)

In [ ]:
np.std(ppo_rews), np.std(td3_rews), np.std(tqc_rews), np.std(reppo_rews), np.std(const_rews)

In [ ]:
np.mean(ppo_rews), np.mean(td3_rews), np.mean(tqc_rews), np.mean(reppo_rews), np.mean(lppo_rew), -m_reward

## Agent Actions Plot

In [ ]:
import os

In [ ]:
config = {
    "w_mort_scale" : 600,
    "growth_k": 0.70,
    'random_start':True,
    'var_penalty_const': 0
    # "curriculum": True
}

In [ ]:
evalEnv =  greenCrabMonthEnvNormalized(config)

In [ ]:
td3Agent = TD3.load(f"varianceRatio0/TD3_gcmenorm", device="cpu")
ppoAgent = PPO.load(f"varianceRatio0/PPO_gcmenorm", device="cpu")
tqcAgent = TQC.load(f"varRatio0.3_new/TQC_gcmenorm", device="cpu")
recurrentPPOAgent = RecurrentPPO.load("varianceRatio0.3/RecurrentPPO_gcmenorm_256_1_varR0.3", device="cpu")
lppoAgent = LipschitzPPO.load("varRatio0.3_new/LipschitzPPO_gcmenorm_0.1_1", device="cpu")

In [ ]:
ppoDataDF = pd.DataFrame(environment_simulation(eval_env, ppoAgent))
DATAPATH = './rl4greencrab/data'
ppoDataDF.to_csv(os.path.join(DATAPATH,f"ppo_env.csv"), index = False)

In [ ]:
ppoDataDF = agent_action_plot(evalEnv=evalEnv, agent=ppoAgent)

In [ ]:
ppoDataDF = agent_action_plot(evalEnv=evalEnv, agent=tqcAgent)

## Generate env simulation data

In [3]:
evalEnv =  greenCrabMonthEnvNormalized(config)

In [4]:
td3Agent = TD3.load(f"varianceRatio0/TD3_gcmenorm", device="cpu")
ppoAgent = PPO.load(f"varianceRatio0/PPO_gcmenorm", device="cpu")
tqcAgent = TQC.load(f"varRatio0.3_new/TQC_gcmenorm", device="cpu")
recurrentPPOAgent = RecurrentPPO.load("varianceRatio0.3/RecurrentPPO_gcmenorm_256_1_varR0.3", device="cpu")
lppoAgent = LipschitzPPO.load("varRatio0.3_new/LipschitzPPO_gcmenorm_0.1_1", device="cpu")

In [5]:
# normalized constant action
def normalized_action(act, max_act):
    max_action = max_act
    constant_act = np.array(act)
    return constant_act * 2 / max_action - 1 

In [6]:
norm_act = normalized_action([0.0, 0.0, 154.3889141214999], 3000)
print(norm_act)
optimal_constant_agent = multiConstAction(env=evalEnv, action=norm_act)
zero_constant_agent = multiConstAction(env=evalEnv, action=np.array([-1, -1, -1]))

[-1.         -1.         -0.89707406]


In [7]:
evalEnv =  greenCrabMonthEnvNormalized(config)
td3_plot_agent = plot_agent(env_sim_df=None, 
                            agent_name='td3_agent_rep500', 
                            env=evalEnv, 
                            agent=td3Agent, 
                            save_dir='.')
td3_plot_agent.gen_env_sim_df(rep=1)
td3_plot_agent.save_df(td3_plot_agent.env_simulation_df, 'td3_sim_500')

/opt/conda/lib/python3.12/site-packages/gymnasium/spaces/box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


In [8]:
evalEnv =  greenCrabMonthEnvNormalized(config)
ppo_plot_agent = plot_agent(env_sim_df=None, 
                            agent_name='ppo_agent_rep500', 
                            env=evalEnv, 
                            agent=ppoAgent, 
                            save_dir='.')
ppo_plot_agent.gen_env_sim_df(rep=1)
ppo_plot_agent.save_df(ppo_plot_agent.env_simulation_df, 'ppo_sim_500')

In [9]:
evalEnv =  greenCrabMonthEnvNormalized(config)
tqc_plot_agent = plot_agent(env_sim_df=None, 
                            agent_name='tqc_agent_rep500', 
                            env=evalEnv, 
                            agent=tqcAgent, 
                            save_dir='.')
tqc_plot_agent.gen_env_sim_df(rep=500)
tqc_plot_agent.save_df(tqc_plot_agent.env_simulation_df, 'tqc_sim_500')

In [ ]:
evalEnv =  greenCrabMonthEnvNormalized(config)
rppo_plot_agent = plot_agent(env_sim_df=None, 
                            agent_name='rppo_agent_rep500', 
                            env=evalEnv, 
                            agent=recurrentPPOAgent, 
                            save_dir='.')
rppo_plot_agent.gen_env_sim_df(rep=500)
rppo_plot_agent.save_df(rppo_plot_agent.env_simulation_df, 'rppo_sim_500')

In [ ]:
evalEnv =  greenCrabMonthEnvNormalized(config)
optimal_constant_plot_agent = plot_agent(env_sim_df=None, 
                            agent_name='optimal_constant_agent_rep500', 
                            env=evalEnv, 
                            agent=optimal_constant_agent, 
                            save_dir='.')
optimal_constant_plot_agent.gen_env_sim_df(rep=1)
optimal_constant_plot_agent.save_df(optimal_constant_plot_agent.env_simulation_df, 'optimal_constant_sim_500')

In [ ]:
evalEnv =  greenCrabMonthEnvNormalized(config)
zero_constant_plot_agent = plot_agent(env_sim_df=None, 
                            agent_name='zero_constant_agent_rep500', 
                            env=evalEnv, 
                            agent=zero_constant_agent, 
                            save_dir='.')
zero_constant_plot_agent.gen_env_sim_df(rep=1)
zero_constant_plot_agent.save_df(zero_constant_plot_agent.env_simulation_df, 'zero_constant_sim_500')

## Rewards plots

In [ ]:
ppo_df = pd.read_csv("rl4greencrab/data/ppo_env.csv")
rppo_df = pd.read_csv("rl4greencrab/data/rppo_env.csv")
td3_df = pd.read_csv("rl4greencrab/data/td3_env.csv")
tqc_df = pd.read_csv("rl4greencrab/data/tqc_env.csv")
constant_df =  pd.read_csv("rl4greencrab/data/constant_agent_env.csv")

In [ ]:
agent_dict = {
    'td3Agent': td3_df,
    'ppoAgent': ppo_df,
    'tqcAgent': tqc_df,
    'recurrentPPOAgent': rppo_df,
    'constantAgent': constant_df
}

In [ ]:
from rl4greencrab import generate_policy_rews_df_dfs
rew_df = generate_policy_rews_df_dfs(agent_dict)

In [ ]:
ppo_plot_agent = plot_agent(env_sim_df=ppo_df, agent_name='ppo_agent', save_dir = './ppo_plot')
tqc_plot_agent = plot_agent(env_sim_df=tqc_df, agent_name='tqc_agent', save_dir = './tqc_plot')

In [ ]:
tqc_plot_agent.agent_action_overtime_plots()
tqc_plot_agent.agent_ob_overtime_plots(obs_name='obs0')

In [ ]:
ppo_plot_agent.agent_action_overtime_plots()
ppo_plot_agent.agent_ob_overtime_plots(obs_name='obs0')
ppo_plot_agent.obs_vs_acts_plots(ob_name='obs1')
ppo_plot_agent.gaussian_smoothing()